# Metrics and Plots for the trained models

This notebook takes trained models objects found in their reosective paths and creats results tables in `results/scores_tables`

> beware, running it might override files if new models are added

In [1]:
seed=0

In [2]:
MAIN_DUMP=f'../../dump_seeds/dump_{seed}'
import os,sys
from joblib import Parallel, delayed,dump,load

sys.path.append(os.path.abspath('../../src/ml'))
from load_matrix import load_df
from model_trainer import MLModel
from utils import load_models

import pandas as pd

from evaluator import Evaluator
from collector import ResultsCollector
from visualizer import DashboardVisualizer


## v2.10

In [3]:
current_version='v2.10'
standard_models=load_models(MAIN_DUMP,version=current_version,normalization='standard')
robust_models=load_models(MAIN_DUMP,version=current_version,normalization='robust')
none_models =load_models(MAIN_DUMP,version=current_version,normalization='none')
log1p_models=load_models(MAIN_DUMP,version=current_version,normalization='log1p')
minmax_models=load_models(MAIN_DUMP,version=current_version,normalization='minmax')

In [4]:
collector = ResultsCollector()
metrics_list = []
metrics_list=[]
for norm in ['none','robust','standard','log1p','minmax']:
        current_dict=eval(f"{norm}_models")
        for model_name, ml_model in current_dict.items():
                print(f'-- Evaluating model: {model_name} {current_version} {norm} --')
                y_test, y_pred, y_proba = ml_model.y_test, ml_model.y_pred, ml_model.y_proba
                metrics = Evaluator(y_test, y_pred, y_proba).compute_metrics()
                metrics_list.append({
                        "model": ml_model.model_type,
                        "input": ml_model.dataset_name,
                        "normalization": norm,  
                        "version": current_version,
                        **metrics
                })

                collector.add(ml_model.model_type, ml_model.dataset_name, y_test, y_pred, y_proba)
results=pd.DataFrame(metrics_list)

# -- gex not related to version
results.loc[results['input']=='gene_expression', 'version'] = ''

-- Evaluating model: pytorch_mlp_Complex_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_Complex_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_concatenated_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_concatenated_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_gene_expression_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_RGCN_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: pytorch_mlp_RGCN_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_Complex_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_Complex_sample_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model: random_forest_concatenated_protein_embeddings_gridsearch_model.joblib v2.10 none --
-- Evaluating model:

In [5]:
results[results['input']=='gene_expression'] # -- checking version

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
4,pytorch_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.906699,0.975409,0.179481
13,random_forest,gene_expression,none,,0.714115,0.860465,0.973684,0.913580,0.545073,0.921053,0.976706,0.105057
26,sklearn_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.500000,0.775510,0.244545
39,svm,gene_expression,none,,0.928230,0.972973,0.947368,0.960000,0.831005,0.983254,0.995719,0.044694
52,xgboost,gene_expression,none,,0.869617,0.945946,0.921053,0.933333,0.717264,0.913876,0.971725,0.087671


In [6]:
results_10=results

In [7]:
results.loc[results['model']=='svm']

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
33,svm,Complex_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.930622,0.980857,0.095701
34,svm,Complex_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.622010,0.863608,0.167282
35,svm,concatenated_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.509569,0.807459,0.174280
36,svm,concatenated_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.538278,0.777521,0.173975
37,svm,GAT_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.901914,0.970470,0.121387
38,svm,GAT_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.892344,0.970153,0.114572
39,svm,gene_expression,none,,0.928230,0.972973,0.947368,0.960000,0.831005,0.983254,0.995719,0.044694
40,svm,GraphSAGE_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.952153,0.987327,0.073590
41,svm,GraphSAGE_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.732057,0.895719,0.160220
42,svm,RGCN_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.509569,0.807459,0.174571


## v2.11

In [8]:
current_version='v2.11'
standard_models=load_models(MAIN_DUMP,version=current_version,normalization='standard')
robust_models=load_models(MAIN_DUMP,version=current_version,normalization='robust')
none_models =load_models(MAIN_DUMP,version=current_version,normalization='none')
log1p_models=load_models(MAIN_DUMP,version=current_version,normalization='log1p')
minmax_models=load_models(MAIN_DUMP,version=current_version,normalization='minmax')

collector = ResultsCollector()
metrics_list = []
metrics_list=[]
for norm in ['none','robust','standard','log1p','minmax']:
        current_dict=eval(f"{norm}_models")
        for model_name, ml_model in current_dict.items():
                print(f'-- Evaluating model: {model_name} {current_version} {norm} --')
                y_test, y_pred, y_proba = ml_model.y_test, ml_model.y_pred, ml_model.y_proba
                metrics = Evaluator(y_test, y_pred, y_proba).compute_metrics()
                metrics_list.append({
                        "model": ml_model.model_type,
                        "input": ml_model.dataset_name,
                        "normalization": norm,  
                        "version": current_version,
                        **metrics
                })

                collector.add(ml_model.model_type, ml_model.dataset_name, y_test, y_pred, y_proba)
results=pd.DataFrame(metrics_list)
results.loc[results['input']=='gene_expression', 'version'] = ''

-- Evaluating model: pytorch_mlp_Complex_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_Complex_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_concatenated_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_concatenated_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_RGCN_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: pytorch_mlp_RGCN_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_Complex_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_Complex_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_concatenated_protein_embeddings_gridsearch_model.joblib v2.11 none --
-- Evaluating model: random_forest_concatenated_sample_embeddings_gridsearch_model.joblib v2.11 none --
-- 

In [9]:
results_11=results

### saving csv

In [10]:
results_10.to_csv(f'../../results/score_tables/scores_v2.10_seed{seed}.csv',index=False)
results_11.to_csv(f'../../results/score_tables/scores_v2.11_seed{seed}.csv',index=False)

all_results=pd.concat([results_10,results_11],axis=0)
all_results.to_csv(f'../../results/score_tables/scores_all_seed{seed}.csv',index=False)

### analysis

In [11]:
from IPython.display import display
display(all_results)

,model,input,normalization,version,balanced_accuracy,precision,recall,f1,mcc,auroc,auprc,brier
0,pytorch_mlp,Complex_protein_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.846890,0.960741,0.196809
1,pytorch_mlp,Complex_sample_embeddings,none,v2.10,0.500000,0.775510,1.000000,0.873563,0.000000,0.342105,0.730382,0.202039
2,pytorch_mlp,concatenated_protein_embeddings,none,v2.10,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.775510,0.775510
3,pytorch_mlp,concatenated_sample_embeddings,none,v2.10,0.507177,0.800000,0.105263,0.186047,0.019786,0.507177,0.778088,0.714286
4,pytorch_mlp,gene_expression,none,,0.500000,0.775510,1.000000,0.873563,0.000000,0.906699,0.975409,0.179481
...,...,...,...,...,...,...,...,...,...,...,...,...
169,xgboost,GraphSAGE_protein_embeddings,minmax,v2.11,0.758373,0.909091,0.789474,0.845070,0.459778,0.904306,0.973010,0.142006
170,xgboost,GraphSAGE_sample_embeddings,minmax,v2.11,0.648325,0.842105,0.842105,0.842105,0.296651,0.779904,0.939636,0.181630
171,xgboost,RGCN_protein_embeddings,minmax,v2.11,0.824163,0.921053,0.921053,0.921053,0.648325,0.940191,0.982707,0.090958
172,xgboost,weighted_RGCN_protein_embeddings,minmax,v2.11,0.869617,0.945946,0.921053,0.933333,0.717264,0.944976,0.985342,0.084233


### averaging

In [12]:
# -- take all seed files and make an average dataframe of all seeds
df0 = pd.read_csv('../../results/score_tables/scores_all_seed0.csv'); df0['seed'] = 0
df1 = pd.read_csv('../../results/score_tables/scores_all_seed1.csv'); df1['seed'] = 1
df2 = pd.read_csv('../../results/score_tables/scores_all_seed2.csv'); df2['seed'] = 2
df3 = pd.read_csv('../../results/score_tables/scores_all_seed3.csv'); df3['seed'] = 3
df4 = pd.read_csv('../../results/score_tables/scores_all_seed4.csv'); df4['seed'] = 4
df5 = pd.read_csv('../../results/score_tables/scores_all_seed5.csv'); df5['seed'] = 5
df6 = pd.read_csv('../../results/score_tables/scores_all_seed6.csv'); df6['seed'] = 6
df7 = pd.read_csv('../../results/score_tables/scores_all_seed7.csv'); df7['seed'] = 7
df8 = pd.read_csv('../../results/score_tables/scores_all_seed8.csv'); df8['seed'] = 8
df9 = pd.read_csv('../../results/score_tables/scores_all_seed9.csv'); df9['seed'] = 9


dfs = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]


# concatenate all seeds
all_df = pd.concat(dfs)

# -- those who have null version => "no version"
all_df['version'] = all_df['version'].fillna('no version').replace('', 'no version')
all_df

avg_df = all_df.groupby(['model','input','normalization','version']).mean().reset_index().drop(columns=['seed'], errors='ignore')
std_df = all_df.groupby(['model','input','normalization','version']).std().reset_index().drop(columns=['seed'], errors='ignore')

In [13]:
# -- remove the seed col
all_df['input']=='gene_expression'

0      False
1      False
2      False
3      False
4       True
       ...  
180    False
181    False
182    False
183    False
184    False
Name: input, Length: 2021, dtype: bool

In [14]:
# -- save all_df
all_df.to_csv('../../results/score_tables/scores_all_seeds.csv',index=False)
avg_df.to_csv('../../results/score_tables/scores_avg_seeds.csv',index=False)
std_df.to_csv('../../results/score_tables/scores_std_seeds.csv',index=False)

In [15]:
avg_df['input'].unique()

array(['Complex_protein_embeddings', 'Complex_sample_embeddings',
       'RGCN_protein_embeddings', 'RGCN_sample_embeddings',
       'concatenated_protein_embeddings',
       'concatenated_sample_embeddings', 'gene_expression',
       'GAT_protein_embeddings', 'GAT_sample_embeddings',
       'GraphSAGE_protein_embeddings', 'GraphSAGE_sample_embeddings',
       'weighted_RGCN_protein_embeddings',
       'weighted_RGCN_sample_embeddings'], dtype=object)